In [1]:
import re
from robobrowser import RoboBrowser
import pandas as pd
from fake_useragent import UserAgent
UserAgent().chrome

'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'

In [2]:
browser = RoboBrowser(user_agent='User_Agent',parser='lxml')
#page_link = 'https://президентскиегранты.рф/public/application/table?CompetitionId=c6559afc-9d8d-49fc-8f00-db7955e29690&RegionId=&ContestDirectionTenantId=&SearchString=&OnlyWinners=true'#ссылка на победителей 2-го конкурса
page_link = 'https://президентскиегранты.рф/public/application/table?CompetitionId=259bc90b-a17d-4ae5-a52e-2d75ce40bf1d&RegionId=&ContestDirectionTenantId=&SearchString=&OnlyWinners=true' #ссылка на победителей 1-го конкурса

In [3]:
# Функция принимает на вход ссылку, 
def page_crawler(page_link):
    browser.open(page_link)
    all_page_links = []
    all_page_links.append(browser.find(class_='pagination__item active').find('a')) 
    for every_next_page in range(int(browser.find(class_='pagination__item active').find_next_siblings()[-1].text.strip('\n'))-1):

        next_page_link = browser.find(class_='pagination__item active').find_next_siblings()[0].find('a')
        all_page_links.append(next_page_link)
        browser.follow_link(all_page_links[-1])

    return all_page_links

In [4]:
#Функция собирает со всех страниц ссылки на проекты
def get_project_links (all_page_links):
    all_project_links = []
    for page_link in all_page_links:
        browser.follow_link(page_link)
        for every_a in browser.find_all(class_='table__link table__link--nopws'):
            all_project_links.append(every_a)
    return all_project_links

In [5]:
# Функция, которая забирает регион из главной таблицы
def get_region(all_page_links):
    region_list = []
    for page_link in all_page_links:
        browser.follow_link(page_link)
        for every_div in browser.find_all(class_='table__cell js-pp-rig-clone-from'):
            string = every_div.text
            match = re.search(r'[А-Я]{1}[а-я]+\s*-*[А-Яа-я]*', string)
            reg = match[0]
            region_list.append(reg)
    return region_list

In [6]:
#Функция, котоая собирает данные со страниц проектов
def get_project_data(all_project_links):
    dataset = []
    for link in all_project_links:
        browser.follow_link(link)
        address = browser.find(class_='winner__details-contacts-item').text # address
        applicationDate = browser.find_all(class_='winner-info__list-item-text')[4].text # applicationDate
        applicationNum = browser.find_all(class_='winner-info__list-item-text')[3].text #applicationNum
        cofunding = browser.find_all(class_='circle-bar__info-item-number')[1].text #cofunding
        contest = browser.find_all(class_='winner-info__list-item-text')[0].text #contest
        socialSignificance = browser.find_all(class_='winner__details-box js-ancor-box')[3].text # socialSignificance
        focus = browser.find_all(class_='winner-info__list-item-text')[1].text # focus
        grantAmount = browser.find_all(class_='circle-bar__info-item-number')[0].text # grantAmount
        operatorINN = '7703424091' # operatorINN
        operatorOGRN = '1177700003942' # operatorOGRN
        operatorName = 'Фонд-оператор президентских грантов по развитию гражданского общества' # operatorName
        time = browser.find_all(class_='winner-info__list-item-text')[5].text # projectBeginning #projectTermination
        projectDescription = browser.find(id='winner-summary').text # projectDescription
        projectGeography = browser.find(id='winner-geography').text # projectGeography
        projectName = browser.find(class_='winner-info__title').text  # projectName
        rating = browser.find_all(class_='winner-info__list-item-text')[2].text # rating
        receiverINN = browser.find_all(class_='winner-info__list-item-text')[7].text # receiverINN
        receiverName = browser.find_all(class_='winner-info__list-item-text')[6].text # receiverName
        receiverOGRN = browser.find_all(class_='winner-info__list-item-text')[8].text # receiverOGRN
        recordUrl = 'https://президентскиегранты.рф' + link['href'] # recordUrl
        
        
        if browser.find(class_='winner__details-contacts-item winner__details-contacts-item--link') == None: # website
            website = 'нет'
        else:
            website = browser.find(class_='winner__details-contacts-item winner__details-contacts-item--link').text

        
        year = '2019'
        full_data = [address, applicationDate,applicationNum, cofunding, 
                         contest, socialSignificance, focus, grantAmount, 
                         operatorINN, operatorOGRN, operatorName, time, projectDescription,
                         projectGeography, projectName, rating,
                         receiverINN, receiverName, receiverOGRN, recordUrl,
                         website, year]
        dataset.append(full_data)
    return dataset

In [7]:
dataset = get_project_data(get_project_links(page_crawler(page_link)))

In [9]:
df = pd.DataFrame(dataset, columns = ['address', 'applicationDate', 'applicationNum', 'cofunding', 
                         'contest', 'socialSignificance', 'focus', 'grantAmount', 
                         'operatorINN', 'operatorOGRN', 'operatorName', 'time', 'projectDescription',
                        'projectGeography', 'projectName', 'rating', 'receiverINN', 
                         'receiverName', 'receiverOGRN', 'recordUrl',
                        ' website', 'year'])

In [10]:
df['region'] = get_region(page_crawler(page_link))

In [11]:
# Чистим applicationDate
applicationDate = []
for every_str in df['applicationDate']:
    every_str = every_str.strip('\n').strip(' ')
    applicationDate.append(every_str)
df['applicationDate'] = applicationDate

In [12]:
# applicationNum
applicationNum = []
for every_str in df['applicationNum']:
    every_str = every_str.strip('\n ').strip(' ')
    applicationNum.append(every_str)
df['applicationNum'] = applicationNum

In [13]:
# cofunding
cofunding = []
for every_str in df['cofunding']:
    every_str = float(every_str.strip(' ').strip(' ₽').replace(',', '.').replace(' ', ''))
    cofunding.append(every_str)
df['cofunding'] = cofunding

In [14]:
# contest
contest = []
for every_str in df['contest']:
    every_str = every_str.strip('\n ')
    contest.append(every_str)
df['contest'] = contest

In [15]:
# socialSignificance
socialSignificance = []
for every_str in df['socialSignificance']:
    every_str = every_str.strip('\nОбоснование социальной значимости\n ').strip('\n')
    socialSignificance.append(every_str)
df['socialSignificance'] = socialSignificance

In [16]:
# focus
focus = []
for every_str in df['focus']:
    every_str = every_str.strip('\n ')
    focus.append(every_str)
df['focus'] = focus

In [17]:
# grantAmount
grantAmount = []
for every_str in df['grantAmount']:
    every_str = float(every_str.strip(' ').strip(' ₽').replace(',', '.').replace(' ',''))
    grantAmount.append(every_str)
df['grantAmount'] = grantAmount

In [18]:
# time: projectBeginning and projectTermination
projectBeginning = []
projectTermination = []
for every_str in df['time']:
    prbeg = every_str.strip('\n ').split(' - ')[0]
    prterm = every_str.strip('\n ').split(' - ')[1]
    projectBeginning.append(prbeg)
    projectTermination.append(prterm)
df['projectBeginning'] = projectBeginning
df['projectTermination'] = projectTermination
del df['time']

In [19]:
# projectDescription
projectDescription = []
for every_str in df['projectDescription']:
    every_str = every_str.replace('\nКраткое описание\n', '').strip('\n ')
    projectDescription.append(every_str)
df['projectDescription'] = projectDescription

In [20]:
# projectGeography
projectGeography = []
for every_str in df['projectGeography']:
    every_str = every_str.replace('\nГеография проекта\n                            ', '').rstrip('\n ')
    projectGeography.append(every_str)
df['projectGeography'] = projectGeography

In [21]:
# rating
rating = []
for every_str in df['rating']:
    every_str = float(every_str.strip('\n ').replace(',', '.'))
    rating.append(every_str)
df['rating'] = rating

In [22]:
# receiverINN
receiverINN = []
for every_str in df['receiverINN']:
    every_str = every_str.strip('\n ')
    receiverINN.append(every_str)
df['receiverINN'] = receiverINN

In [23]:
# receiverName
receiverName = []
for every_str in df['receiverName']:
    every_str = every_str.strip('\n ')
    receiverName.append(every_str)
df['receiverName'] = receiverName

In [24]:
# receiverOGRN
receiverOGRN = []
for every_str in df['receiverOGRN']:
    every_str = every_str.strip('\n ')
    receiverOGRN.append(every_str)
df['receiverOGRN'] = receiverOGRN

In [25]:
df.to_csv('1202_grants_1.csv', sep=',', encoding = 'utf-8-sig')
#df.to_csv('1202_grants.csv', sep=',', encoding = 'utf-8-sig')

In [ ]:
#соединяем собранные датасеты с разных конкурсов и датасет 2013-2018 гг.
df1 = pd.read_csv('1202_grants_1.csv', sep=',', encoding = 'utf-8-sig')
df2 = pd.read_csv('1202_grants.csv', sep=',', encoding = 'utf-8-sig')
df3 = pd.read_csv('no_email_tel_issign.csv', sep = ',', encoding = 'utf-8-sig')

In [ ]:
frames = [df1, df2,df3]
result = pd.concat(frames)
result = pd.concat(frames, sort=True)
result.to_csv('GRANT_2013_2019.csv', sep = ',', encoding = 'utf-8-sig')